# llama api (더 정확한 RAG 데이터 생성을 위해 api사용)

In [ ]:
from pathlib import Path
import nest_asyncio
from llama_parse import LlamaParse # llama api이용 (RAG 데이터)
import nest_asyncio

PDF_DIR = Path("../pdfs")
OUT_DIR = Path("../outputs")

assert PDF_DIR.exists(), f"PDF_DIR 경로를 찾을 수 없습니다. : {PDF_DIR.resolve()}"
pdfs = sorted(PDF_DIR.glob("*.pdf"))
if not pdfs:
    print(f"경로에 PDF가 없습니다. {PDF_DIR.resolve()}")


OUT_DIR.mkdir(parents=True, exist_ok=True)
nest_asyncio.apply()
for p in pdfs:
    parser = LlamaParse(
        api_key="<api_key>",
        parse_mode="parse_page_with_agent",
        model="anthropic-sonnet-4.0",
        high_res_ocr=True,
        adaptive_long_table=True,
        outlined_table_extraction=True,
        output_tables_as_HTML=True,
        language="ko",
        result_type="markdown",
        page_separator="\n=================\n",
    )

    print(f"{p.name} 변환 시작")
    full_file = ""
    documents = parser.load_data(p)

    for i in range(len(documents)):
        full_file += f"**=========={i + 1} 페이지==========**\n" + documents[i].text + "\n\n"

    output_path  = OUT_DIR / f"{p.stem}.md"
    output_path.write_text(full_file, encoding="utf-8")

# 무료 라이브러리 (실제 프로젝트 설계시 사용)

In [ ]:
import pymupdf4llm # pip 필요

# PDF에서 텍스트 추출
md_text = pymupdf4llm.to_markdown("marketing_plan_Stratsy.pdf")

md_text # 이 텍스트를 내부 벡터db저장, md_text와 사용자 텍스트를 합치고 모델 인풋으로